<a href="https://colab.research.google.com/github/merongedrago/ML_Cybersecurity/blob/main/Assignment_1_Colab_and_ML_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1: Colab and ML basics

**Name:** Meron Gedrago

---

## Instructions
1. **IMPORTANT:** Make sure to `Save a Copy` in your local drive or to `download` this notebook on your local machine.
1. **Don't Panic:** We will walk through this step-by-step.
2.  **Run in Order:** Make sure you run the cells from top to bottom.
3.  **Look for the Comments:** Wherever you see `### YOUR CODE HERE ###`, that is where you need to type.


---
***Attribution**: This notebook was created by Michael Roman for use in the CYBERSEC 520 course a DUKE University.  This notebook has been adapted from Lecture notes. Gemini 3-Pro was used to format and clean up this notebook.*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mproman/CYBERSEC520/blob/main/content/assignments/Assignment_1.ipynb)

## Step 0: Notebook Setup
We need to bring in our tools. These are the standard libraries used by Data Scientists everywhere.
* **Pandas:** For handling data like Excel sheets.
* **Seaborn/Matplotlib:** For drawing graphs.
* **Sklearn:** The Machine Learning library.

**Action:** Click the "Play" button on the cell below.

In [44]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import Machine Learning tools
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn.svm import SVC
# calculate the baseline f1 score
from sklearn.metrics import f1_score
# Make the graphs look nice
plt.style.use('seaborn-v0_8-whitegrid')
print("Setup Complete! You are ready to start.")

Setup Complete! You are ready to start.


## Step 1: Testing with Penguins
Before we tackle a Cyber dataset, let's make sure everything works with a simple dataset: **The Palmer Penguins**.

**Action:** Run the cell below to load the data.

In [18]:
# Load the data from a URL
url = "https://github.com/allisonhorst/palmerpenguins/raw/5b5891f01b52ae26ad8cb9755ec93672f49328a8/data/penguins_size.csv"
penguins = pd.read_csv(url)

# Drop missing values to prevent errors
penguins = penguins.dropna()

# We will just use two math features for the warm up: Bill Length and Flipper Length
# We are trying to predict the 'species'
X_warmup = penguins[['culmen_length_mm', 'flipper_length_mm']]
y_warmup = penguins['species_short']

# Split into Train and Test (80% training, 20% testing)
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_warmup, y_warmup, test_size=0.2, random_state=42)

print("Penguin data loaded and split successfully.")
X_warmup.head()

Penguin data loaded and split successfully.


,culmen_length_mm,flipper_length_mm
0,39.1,181.0
1,39.5,186.0
2,40.3,195.0
4,36.7,193.0
5,39.3,190.0


### 1.1 Train a Simple Model
Now, we will train a **K-Nearest Neighbors (k-NN)** model.
* **Action:** Change `n_neighbors=5` to a different number (like 1, 5 or 7) if you want, and run the cell.

In [19]:
# 1. Create the model
# You can change n_neighbors to see what happens
model = KNeighborsClassifier(n_neighbors=3)

# 2. Train the model (The "Learning" part)
model.fit(X_train_w, y_train_w)

# 3. Predict on the test set
predictions = model.predict(X_test_w)

# 4. Check the score
score = accuracy_score(y_test_w, predictions)
print(f"Warmup Accuracy: {score:.2f} (or {score*100:.1f}%)")

Warmup Accuracy: 1.00 (or 100.0%)


## Step 2: Cyber Data
Now that we know the code works, let's use the real assignment data.

**Instructions:**
1.  Download the dataset from Canvas or import directly from Github.
2.  If using **Google Colab**: Click the folder icon on the left, then the upload icon (page with an arrow), and upload your `.csv` file. If you want to save the file in your drive, make sure you `mount` your drive by running the cell below.
3.  **Action:** Update the `filename` variable below to match your file's name.

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
# Update this string to match your filename!

path ="drive/My Drive/CyberSecurity_Datasets/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv"
#note - if you upload to google drive it may read something like this:
#path = "drive/My Drive/CYBERSEC520/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"

# Load the data
# We use 'try' and 'except' in case the file isn't found
try:
    df = pd.read_csv(path)
    print("Data loaded successfully!")
except FileNotFoundError:
    print("ERROR: File not found. Did you upload it? Did you spell the name right?")

Data loaded successfully!


### 2.1 Clean and Subsample
Cyber datasets are **huge**. To speed up your runs, we will take a random sample (a smaller slice) of the data.

In [77]:
# 1. Clean up weird values (Infinite numbers and empty spots)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# 2. Subsample
# n=2000 means you will take a random selection of 2000. random_state=42 fixes the random seed for repeatability

df_small = df.sample(n=2000, random_state=42)

print(f"Original size: {len(df)} rows")
print(f"New size: {len(df_small)} rows")

Original size: 286096 rows
New size: 2000 rows


### 2.2 Prepare the Data
We need to separate the **Features** (the math/stats columns) from the **Label** (Benign vs. Attack).

In [78]:
# For the dataset in class, the column name for the target is ' Label' (with a space) make sure this maps to your dataset 'Label'
# Let's check the columns first
print(df_small.columns)

# Define X (Features) and y (Target)
# We drop the label from X
X = df_small.drop(columns=[' Label'])
y = df_small[' Label']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Cyber data is ready for training.")

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

## Step 3: Baseline Model
We will create a "Baseline". This is the model with default settings. We need this number to see if we can improve it later.

**Action:** Fill in the code below. Use `KNeighborsClassifier`, `SVC`, or another model if you learned one you like better.

In [80]:
# --- STUDENT AREA ---

# 1. Initialize the model (Use default settings, leave the parentheses empty)
# Hint: clf = KNeighborsClassifier() or clf = SVC()
clf_baseline = KNeighborsClassifier()   ### YOUR CODE HERE ###


# 2. Fit the model on X_train and y_train
clf_baseline.fit(X_train, y_train)      ### YOUR CODE HERE ###

# 3. Predict on X_test
y_pred_base = clf_baseline.predict(X_test) ### YOUR CODE HERE ###

# --- END STUDENT AREA ---

# Calculate Accuracy
baseline_acc = f1_score(y_test, y_pred_base, pos_label="PortScan")
print(f"Baseline Accuracy: {baseline_acc:.4f}")

Baseline Accuracy: 0.9676


## Step 4: Hyperparameter Tuning
Now, play the role of the Scientist. Change the settings (Hyperparameters) to try and beat your baseline score.

If you are using k-NN, try changing:
* `n_neighbors` (Try 1, 3, 10, 50)
* `weights` (Try 'uniform' vs 'distance')

If you are using SVM, try changing:
* `C` (Try 0.1, 1.0, 10, 100 etc. )
* `kernel` (Try `linear`, `poly`, or `rbf`)
* `gamma` (Try  1.0, 0.1, 0.01, etc.)

*Note: **gamma** is only used for `rbf`, `poly`, and `sigmoid` kernels.*


In [81]:
# implementing a gridsearch to tune the hyperparameter to find optimal hyperparameters
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

knn = KNeighborsClassifier()
param_grid = {
    'n_neighbors': range(1, 20), # Test k from 1 to 20
    'weights': ['uniform', 'distance']
}


grid_search = RandomizedSearchCV(estimator=knn, param_distributions=param_grid,
                                 cv=5,
                                 n_iter=10, scoring='accuracy',
                                 random_state=42,
                                 return_train_score=False,
                                 verbose=10)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START n_neighbors=17, weights=distance...........................
[CV 1/5; 1/10] END n_neighbors=17, weights=distance;, score=0.978 total time=   0.0s
[CV 2/5; 1/10] START n_neighbors=17, weights=distance...........................
[CV 2/5; 1/10] END n_neighbors=17, weights=distance;, score=0.972 total time=   0.0s
[CV 3/5; 1/10] START n_neighbors=17, weights=distance...........................
[CV 3/5; 1/10] END n_neighbors=17, weights=distance;, score=0.966 total time=   0.0s
[CV 4/5; 1/10] START n_neighbors=17, weights=distance...........................
[CV 4/5; 1/10] END n_neighbors=17, weights=distance;, score=0.953 total time=   0.0s
[CV 5/5; 1/10] START n_neighbors=17, weights=distance...........................
[CV 5/5; 1/10] END n_neighbors=17, weights=distance;, score=0.969 total time=   0.0s
[CV 1/5; 2/10] START n_neighbors=19, weights=uniform............................
[CV 1/5; 2/10] END n_neighbo

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(),
                   param_distributions={'n_neighbors': range(1, 20),
                                        'weights': ['uniform', 'distance']},
                   random_state=42, scoring='accuracy', verbose=10)

In [82]:
print(grid_search.best_params_)

print(grid_search.best_estimator_)

{'weights': 'uniform', 'n_neighbors': 3}
KNeighborsClassifier(n_neighbors=3)


In [83]:
# --- EXPERIMENT 1 ---

# Change the parameters inside the parentheses

    # If you are sticking with k-NN change n_neighbors (try 1, 3, 11) or weights ('uniform', 'distance')
    # Example: clf_experiment = KNeighborsClassifier(n_neighbors=11, weights='distance')

    #If you are using the SVC (Support Vector Machine Classifier) try changing C (0.1, 1, 10), kernel ('linear', 'poly', 'rbf'), or gamma (0.1, 0.01)
    # Note: gamma is only used for 'rbf', 'poly', and 'sigmoid' kernels.
    # Example: clf_experiment = SVC(C=1.0, kernel='rbf', gamma=0.1)

clf_experiment = KNeighborsClassifier(n_neighbors=3, weights='uniform')

# Train
clf_experiment.fit(X_train, y_train)

# Predict
y_pred_exp = clf_experiment.predict(X_test)

# Score
exp_acc = f1_score(y_test, y_pred_exp, pos_label='PortScan')
print(f"Experiment F1 Accuracy: {exp_acc:.4f}")

# Compare
if exp_acc > baseline_acc:
    print("🎉 Success! You improved the model!")
else:
    print("🤔 Hmmm, the baseline was better. Try changing the number again.")

Experiment F1 Accuracy: 0.9719
🎉 Success! You improved the model!


In [85]:
print (f"Baseline accuracy: {accuracy_score(y_test,y_pred_base):.4f}")
print (f"Experiment accuracy: {accuracy_score(y_test,y_pred_exp):.4f}")

Baseline accuracy: 0.9625
Experiment accuracy: 0.9675


## Step 5: Documentation & Reflection 📝
*Double click this cell to edit it and answer the questions.*

**1. What model and dataset did you choose?**
* Dataset: Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
* Model:  k-Nearest Neighbors

**2. What was your baseline performance?**
* Accuracy: 0.9625
* F1_score: 0.9676

**3. What hyperparameter(s) did you change?**
| Trial    | n_neighbors | weights  | F1_score (CV mean) |
| -------- | ----------- | -------- | ------------------ |
| Trial 1  | 3           | uniform  | 0.977              |
| Trial 2  | 4           | uniform  | 0.976              |
| Trial 3  | 7           | distance | 0.974              |
| Trial 4 | 8           | distance | 0.974              |
| Trial 5  | 13          | uniform  | 0.957              |
| Trial 6  | 14          | uniform  | 0.955              |
| Trial 7  | 14          | distance | 0.968              |
| Trial 8  | 16          | uniform  | 0.951              |
| Trial 9  | 17          | distance | 0.968              |
| Trial 10  | 19          | uniform  | 0.949              |




**4. What setting worked the best?**
* The best setting was: weight of 'uniform' and n_neighbours of 3

**5. Hypothesize why you think that is.**
* Overall, we see a trend of decreasing F1 score as the n_neighbors increase, which could be due to different reasons. I think one reason could be that there are tighter clusters, where the closest three neighbors are the best indicator for a given point, compared to widening the group and including observations from a different 'cluster'. The 'uniform' weighting could be because those three points do not outweigh each other in importance and influence, so they should be weighted uniformly rather than by distance.

**6. What issues did you run into? Were you able to solve them?**
* My model threw an error when running calculating F1_score, so I had to explicitly map the positive label to `PortScan` and it also threw errors when I had previously tried a different dataset that had multiple labels more than 2.

## Submission Checklist
* [ ] Did you include your name?
* [ ] Did you list your teammates (if any)?
* [ ] Did you run all cells?
* [ ] Did you answer the reflection questions in Step 5?

**To Submit:** File -> Download -> Download .ipynb